## Visualizing equivalent pathways in PathMe Viewer

As Reactome is hierarchically structured, subpathways of pathways must be accessed. Thus, here URLs for all merged networks of all equivalent pathways (including those Reactome subpathways) are generated.

In [1]:
import collections
import pandas as pd

from bio2bel_reactome import Manager

In [2]:
reactome_manager = Manager()

In [3]:
prefix = 'http://0.0.0.0:5000/pathme/viewer?'
reactome_suffix = 'resources[]=reactome&pathways[]=%s'
kegg_suffix = 'resources[]=kegg&pathways[]=%s'
wikipathways_suffix = 'resources[]=wikipathways&pathways[]=%s'

In [4]:
reactome_ids = ['R-HSA-5358508','R-HSA-209968','R-HSA-195721','R-HSA-5683057','R-HSA-71336','R-HSA-198203','R-HSA-168898','R-HSA-983705','R-HSA-157118','R-HSA-109581','R-HSA-428157','R-HSA-5358351','R-HSA-71403','R-HSA-69306','R-HSA-5693571','R-HSA-1640170','R-HSA-9006936','R-HSA-165159','R-HSA-448424','R-HSA-74182','R-HSA-1170546']               
kegg_ids = ['hsa03430','hsa04918','hsa04310','hsa04010','hsa00030','hsa04151','hsa04620','hsa04662','hsa04330','hsa04210','hsa00600','hsa04340','hsa00020','hsa03030','hsa03450','hsa04110','hsa04350','hsa04150','hsa04657','hsa00072','hsa04917']
wikipathways_ids = ['WP3381','WP1981','WP363','WP382','WP134','WP4172','WP75','WP2746','WP268','WP254','WP2788','WP47','WP78','WP466','WP438','WP179','WP366','WP1471','WP2112','WP311','WP2678']

In [5]:
def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

In [6]:
def get_all_pathway_children_by_id(manager, reactome_id):
    
    pathway = manager.get_pathway_by_id(reactome_id)

    if not pathway.children:
        return pathway.reactome_id
    
    children = []
    
    for child in pathway.children:

        children.append(get_all_pathway_children_by_id(manager, child.reactome_id))
    
    return children

In [7]:
parent_to_child = dict()

for reactome_id in reactome_ids:
    
    all_children = get_all_pathway_children_by_id(reactome_manager, reactome_id)

    if isinstance(all_children, str):
        continue
        
    flattened_children = flatten(all_children)
    parent_to_child[reactome_id] = [pathway for pathway in flattened_children]

16 Reactome pathways contain subpathways and therefore we concatenate each subpathway into the url

In [8]:
len(parent_to_child)

16

In [9]:
merged_pathway_urls = []

for counter, reactome_id in enumerate(reactome_ids):
        
    base_url = prefix+reactome_suffix % reactome_id +'&'+kegg_suffix % (kegg_ids[counter])+'&'+wikipathways_suffix % (wikipathways_ids[counter])

    if reactome_id in parent_to_child:
        
        pathway_children = parent_to_child[reactome_id]
        
        base_url += '&'+'&'.join(
            [reactome_suffix % reactome_id
             for reactome_id in pathway_children]
        )
        
    merged_pathway_urls.append(base_url)

Table with database identifiers and URLS to visualize merged pathways in the PathMe Viewer

In [11]:
pd.set_option('display.max_colwidth', -1)
pd.DataFrame(
    {'KEGG': kegg_ids, 
     'Reactome': reactome_id, 
     'WikiPathways': wikipathways_ids,
     'Merged Pathways': merged_pathway_urls
    },
) 

,KEGG,Reactome,WikiPathways,Merged Pathways
0,hsa03430,R-HSA-1170546,WP3381,http://0.0.0.0:5000/pathme/viewer?resources[]=reactome&pathways[]=R-HSA-5358508&resources[]=kegg&pathways[]=hsa03430&resources[]=wikipathways&pathways[]=WP3381&resources[]=reactome&pathways[]=R-HSA-5358606&resources[]=reactome&pathways[]=R-HSA-5358565
1,hsa04918,R-HSA-1170546,WP1981,http://0.0.0.0:5000/pathme/viewer?resources[]=reactome&pathways[]=R-HSA-209968&resources[]=kegg&pathways[]=hsa04918&resources[]=wikipathways&pathways[]=WP1981&resources[]=reactome&pathways[]=R-HSA-350864
2,hsa04310,R-HSA-1170546,WP363,http://0.0.0.0:5000/pathme/viewer?resources[]=reactome&pathways[]=R-HSA-195721&resources[]=kegg&pathways[]=hsa04310&resources[]=wikipathways&pathways[]=WP363&resources[]=reactome&pathways[]=R-HSA-4086398&resources[]=reactome&pathways[]=R-HSA-4608870&resources[]=reactome&pathways[]=R-HSA-5140745&resources[]=reactome&pathways[]=R-HSA-5099900&resources[]=reactome&pathways[]=R-HSA-196299&resources[]=reactome&pathways[]=R-HSA-4641265&resources[]=reactome&pathways[]=R-HSA-3769402&resources[]=reactome&pathways[]=R-HSA-4641257&resources[]=reactome&pathways[]=R-HSA-4641258&resources[]=reactome&pathways[]=R-HSA-4641262&resources[]=reactome&pathways[]=R-HSA-4411364&resources[]=reactome&pathways[]=R-HSA-5368598&resources[]=reactome&pathways[]=R-HSA-3772470&resources[]=reactome&pathways[]=R-HSA-4641263&resources[]=reactome&pathways[]=R-HSA-201688&resources[]=reactome&pathways[]=R-HSA-3238698
3,hsa04010,R-HSA-1170546,WP382,http://0.0.0.0:5000/pathme/viewer?resources[]=reactome&pathways[]=R-HSA-5683057&resources[]=kegg&pathways[]=hsa04010&resources[]=wikipathways&pathways[]=WP382&resources[]=reactome&pathways[]=R-HSA-112411&resources[]=reactome&pathways[]=R-HSA-110056&resources[]=reactome&pathways[]=R-HSA-5674135&resources[]=reactome&pathways[]=R-HSA-5674499&resources[]=reactome&pathways[]=R-HSA-5673000&resources[]=reactome&pathways[]=R-HSA-5658442&resources[]=reactome&pathways[]=R-HSA-5687128
4,hsa00030,R-HSA-1170546,WP134,http://0.0.0.0:5000/pathme/viewer?resources[]=reactome&pathways[]=R-HSA-71336&resources[]=kegg&pathways[]=hsa00030&resources[]=wikipathways&pathways[]=WP134&resources[]=reactome&pathways[]=R-HSA-73843
5,hsa04151,R-HSA-1170546,WP4172,http://0.0.0.0:5000/pathme/viewer?resources[]=reactome&pathways[]=R-HSA-198203&resources[]=kegg&pathways[]=hsa04151&resources[]=wikipathways&pathways[]=WP4172
6,hsa04620,R-HSA-1170546,WP75,http://0.0.0.0:5000/pathme/viewer?resources[]=reactome&pathways[]=R-HSA-168898&resources[]=kegg&pathways[]=hsa04620&resources[]=wikipathways&pathways[]=WP75&resources[]=reactome&pathways[]=R-HSA-5686938&resources[]=reactome&pathways[]=R-HSA-168142&resources[]=reactome&pathways[]=R-HSA-168179&resources[]=reactome&pathways[]=R-HSA-166058&resources[]=reactome&pathways[]=R-HSA-168927&resources[]=reactome&pathways[]=R-HSA-9014325&resources[]=reactome&pathways[]=R-HSA-9013973&resources[]=reactome&pathways[]=R-HSA-9013957&resources[]=reactome&pathways[]=R-HSA-936964&resources[]=reactome&pathways[]=R-HSA-937041&resources[]=reactome&pathways[]=R-HSA-937072&resources[]=reactome&pathways[]=R-HSA-2562578&resources[]=reactome&pathways[]=R-HSA-166020&resources[]=reactome&pathways[]=R-HSA-937039&resources[]=reactome&pathways[]=R-HSA-937042&resources[]=reactome&pathways[]=R-HSA-450321&resources[]=reactome&pathways[]=R-HSA-450341&resources[]=reactome&pathways[]=R-HSA-199920&resources[]=reactome&pathways[]=R-HSA-202670&resources[]=reactome&pathways[]=R-HSA-450302&resources[]=reactome&pathways[]=R-HSA-445989&resources[]=reactome&pathways[]=R-HSA-975110&resources[]=reactome&pathways[]=R-HSA-975144&resources[]=reactome&pathways[]=R-HSA-975163&resources[]=reactome&pathways[]=R-HSA-168138&resources[]=reactome&pathways[]=R-HSA-1679131
7,hsa04662,R-HSA-1170546,WP2746,http://0.0.0.0:5000/pathme/viewer?resources[]=reactome&pathways[]=R-HSA-983705&resources[]=kegg&pathways[]=hsa04662&resources[]=wikipathways&pathways[]=WP2746&resources[]=reactome&pathw